In [33]:
import pandas as pd
import numpy as np
import pickle

In [34]:
kobe_train = pd.read_csv('Kobe/training_data.csv')

In [35]:
train_numbers = kobe_train.drop(['shot_id','period','playoffs','lat','lon','game_date', 'season', 'shot_type', 'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'opponent', 'Home_vs_Away'], axis=1)

y = train_numbers['shot_made_flag']

train_numbers.drop(['shot_made_flag'], axis= 1, inplace=True)

In [36]:
action_columns = pd.get_dummies(train_numbers['action_type'],prefix = "action")
train_numbers = pd.concat([train_numbers, action_columns], axis=1)
train_numbers.drop('action_type', axis=1, inplace=True)
train_numbers.head()

,loc_x,loc_y,total_seconds_remaining,shot_distance,action_Alley Oop Dunk Shot,action_Alley Oop Layup shot,action_Cutting Layup Shot,action_Driving Bank shot,action_Driving Dunk Shot,action_Driving Finger Roll Layup Shot,...,action_Running Tip Shot,action_Slam Dunk Shot,action_Step Back Jump shot,action_Tip Layup Shot,action_Tip Shot,action_Turnaround Bank shot,action_Turnaround Fadeaway shot,action_Turnaround Finger Roll Shot,action_Turnaround Hook Shot,action_Turnaround Jump Shot
0,-157,0,622,15,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,138,175,412,22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-145,-11,572,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-33,125,216,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-94,238,116,25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
train_numbers.drop(['action_Cutting Layup Shot','action_Driving Floating Bank Jump Shot',
           'action_Hook Bank Shot','action_Putback Slam Dunk Shot','action_Running Finger Roll Shot',
           'action_Running Slam Dunk Shot','action_Tip Layup Shot','action_Turnaround Finger Roll Shot'],axis=1,inplace =True)



In [38]:
train_numbers.head()

,loc_x,loc_y,total_seconds_remaining,shot_distance,action_Alley Oop Dunk Shot,action_Alley Oop Layup shot,action_Driving Bank shot,action_Driving Dunk Shot,action_Driving Finger Roll Layup Shot,action_Driving Finger Roll Shot,...,action_Running Pull-Up Jump Shot,action_Running Reverse Layup Shot,action_Running Tip Shot,action_Slam Dunk Shot,action_Step Back Jump shot,action_Tip Shot,action_Turnaround Bank shot,action_Turnaround Fadeaway shot,action_Turnaround Hook Shot,action_Turnaround Jump Shot
0,-157,0,622,15,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,138,175,412,22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-145,-11,572,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-33,125,216,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-94,238,116,25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel

In [40]:
parameter_grid = {
                 'max_depth' : [4],
                 'n_estimators': [50],
                 'max_features': ['log2'],
                 'min_samples_split': [7],
                 'min_samples_leaf': [4],
                 }
boost = GradientBoostingClassifier()
cross_validation = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(boost,
                           scoring='accuracy',
                           param_grid=parameter_grid,
                           cv=cross_validation,
                           verbose=1,
                           n_jobs=-1
                          )
grid_search.fit(train_numbers, y)
model = grid_search
parameters = grid_search.best_params_
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    6.4s remaining:    9.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.3s finished


Best score: 0.6794956609720979
Best parameters: {'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 50}


In [12]:
# save_model = open("gradient_boosting2.pickle","wb")
# pickle.dump(grid_search, save_model)
# save_model.close()

In [5]:
# classifier_f = open("gradient_boosting2.pickle","rb")
# grid_search = pickle.load(classifier_f)
# classifier_f.close()

EOFError: Ran out of input

In [41]:
shots_test = pd.read_csv('Kobe/test_data.csv')
shots_test.head()

,shot_id,game_date,action_type,loc_x,loc_y,total_seconds_remaining,period,playoffs,season,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,opponent,Home_vs_Away,lat,lon
0,1,10/31/2000,Jump Shot,167,72,627,1,0,2000-01,18,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,POR,Away,33.9723,-118.1028
1,8,10/31/2000,Jump Shot,1,28,485,3,0,2000-01,2,NaN,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,POR,Away,34.0163,-118.2688
2,17,11/1/2000,Driving Layup Shot,0,0,1,1,0,2000-01,0,NaN,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,UTA,Home,34.0443,-118.2698
3,20,11/1/2000,Driving Layup Shot,0,0,646,3,0,2000-01,0,NaN,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,UTA,Home,34.0443,-118.2698
4,33,11/4/2000,Jump Shot,163,76,686,1,0,2000-01,17,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,VAN,Away,33.9683,-118.1068


In [42]:
shots_test.drop(["lat","lon","shot_id","game_date","period","season","shot_type","shot_zone_area","shot_zone_basic","shot_zone_range","Home_vs_Away","playoffs","opponent"],axis=1,inplace=True)
shots_test.head()

,action_type,loc_x,loc_y,total_seconds_remaining,shot_distance,shot_made_flag
0,Jump Shot,167,72,627,18,NaN
1,Jump Shot,1,28,485,2,NaN
2,Driving Layup Shot,0,0,1,0,NaN
3,Driving Layup Shot,0,0,646,0,NaN
4,Jump Shot,163,76,686,17,NaN


In [43]:
action_columns = pd.get_dummies(shots_test['action_type'],prefix = "action")
shots_test = pd.concat([shots_test, action_columns], axis=1)
shots_test.drop('action_type', axis=1, inplace=True)
shots_test.head()

,loc_x,loc_y,total_seconds_remaining,shot_distance,shot_made_flag,action_Alley Oop Dunk Shot,action_Alley Oop Layup shot,action_Cutting Finger Roll Layup Shot,action_Driving Bank shot,action_Driving Dunk Shot,...,action_Running Reverse Layup Shot,action_Running Tip Shot,action_Slam Dunk Shot,action_Step Back Jump shot,action_Tip Shot,action_Turnaround Bank shot,action_Turnaround Fadeaway Bank Jump Shot,action_Turnaround Fadeaway shot,action_Turnaround Hook Shot,action_Turnaround Jump Shot
0,167,72,627,18,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,28,485,2,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,646,0,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,163,76,686,17,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
shots_test.drop('shot_made_flag',axis=1,inplace=True)
shots_test.drop('action_Cutting Finger Roll Layup Shot',axis=1,inplace=True)
shots_test.drop('action_Turnaround Fadeaway Bank Jump Shot',axis=1,inplace=True)
shots_test.head()

,loc_x,loc_y,total_seconds_remaining,shot_distance,action_Alley Oop Dunk Shot,action_Alley Oop Layup shot,action_Driving Bank shot,action_Driving Dunk Shot,action_Driving Finger Roll Layup Shot,action_Driving Finger Roll Shot,...,action_Running Pull-Up Jump Shot,action_Running Reverse Layup Shot,action_Running Tip Shot,action_Slam Dunk Shot,action_Step Back Jump shot,action_Tip Shot,action_Turnaround Bank shot,action_Turnaround Fadeaway shot,action_Turnaround Hook Shot,action_Turnaround Jump Shot
0,167,72,627,18,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,28,485,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,646,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,163,76,686,17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
output = grid_search.predict_proba(shots_test).astype(float)

In [46]:
dummy=pd.read_csv('Kobe/test_data.csv')
submission=pd.DataFrame()
submission['shot_id']=dummy['shot_id']
submission.head()
   

,shot_id
0,1
1,8
2,17
3,20
4,33


In [47]:
submission['shot_made_flag']=0

In [48]:
lists = []
for result in output:
    lists.append(result[0])

In [49]:
submission['shot_made_flag'] = lists

In [50]:
submission.tail()

,shot_id,shot_made_flag
4995,30669,0.639366
4996,30681,0.547515
4997,30683,0.337323
4998,30687,0.302922
4999,30694,0.547515


In [51]:
submission.to_csv('gradient_boosting_test.csv',index=False)